<a href="https://colab.research.google.com/github/dohyun93/hongong_mldl/blob/main/7_2_%EC%8B%AC%EC%B8%B5_%EC%8B%A0%EA%B2%BD%EB%A7%9D.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [11]:
# 7-1 장에서 살펴본 인공신경망은 히든레이어가 1개인 신경망이었다.
# 입력층과 출력층 사이에 층을 더 추가해서 심층 신경망을 만들어보자.

from tensorflow import keras
(train_input, train_target), (test_input, test_target) =\
keras.datasets.fashion_mnist.load_data()

In [12]:
from sklearn.model_selection import train_test_split

train_scaled = train_input / 255.0 # 1. 표준화
train_scaled = train_scaled.reshape(-1, 28*28) # 2. 각 그림을 1차원 벡터로 만듬.

# 3. 훈련 세트에서 다시 훈련/검증 세트로 나눔.
train_scaled, val_scaled, train_target, val_target = train_test_split(train_scaled, train_target, test_size=0.2, random_state=42)

In [13]:
# 이제 인공신경망 모델에 층을 추가해보자.

# 입력층: (n_samples, 28*28)
# h1: (28*28, 100) -> 뉴런 수가 100개인 히든레이어 h1. 활성화함수는 시그모이드/렐루 같은걸 사용할 수 있음.
# 출력층: (100, 10) -> 다중 분류이므로 소프트맥스 함수를 활성화 함수로 적용. 이진분류는 시그모이드함수.

# 회귀의 경우 출력층에 활성화함수를 적용하지 않는다. 그 이유는 분류는 활성화함수로 확률로의 변환이 필요하지만, 회귀는 
# 임의의 숫자를 추론하는 모델이므로 적용하지 않는다.

# 여기서 은닉층에 활성화함수를 사용하는 이유를 살펴보면 아래 예시를 확인해보자.
# a * 3 - 1 = b
# 2 * b + 3 = c 라고 했을 때, 활성화 함수를 적용하지 않는 경우라고 볼 수 있다.
# 결국 c = 6 * a + 1 이되며 b가 하는 일이 없는거나 마찬가지다.
# 이러한 선형 방정식을 적절히 비선형으로 뒤틀어주어야 한다. 그래야 이전 히든레이어가 존재하는 의미가 생기기 때문이다.

# a * 3 - 1 = b
# log(b) = k
# 2 * k + 3 = c
# 처럼.

# 자 그럼 이제 은닉층의 활성화함수로 시그모이드 함수를, 출력층의 활성화 함수로 소프트맥스 함수를 사용한 인공신경망을 만들어보자.
# 히든레이어부터 출력층까지 만든다.
dense1 = keras.layers.Dense(100, activation='sigmoid', input_shape=(28*28,))
dense2 = keras.layers.Dense(10, activation='softmax')

# dense1, dense2 객체를 Sequential 클래스에 추가하여 심층 신경망을 만들어보자.
model = keras.Sequential([dense1, dense2])
# keras는 모델의 summary() 메서드로 층에 대한 유용한 정보를 보여준다.
print(model.summary())

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 100)               78500     
                                                                 
 dense_1 (Dense)             (None, 10)                1010      
                                                                 
Total params: 79,510
Trainable params: 79,510
Non-trainable params: 0
_________________________________________________________________
None
